# Extracting information from a shared chatGPT chat

For background see this discussion comment: [Accessing OpenAI Chat Data and Shared Conversations
](https://github.com/thorwhalen/oa/discussions/11#discussioncomment-11852719).

## Demo

In [31]:
from oa.chats import url_to_html, parse_shared_chatGPT_chat

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

## Bootstrapping the parsing (and maintaining the parser)

If you have a look at the `parse_shared_chatGPT_chat` you'll notice it's parametrized by several arguments you didn't have to specify.

```python
def parse_shared_chatGPT_chat(
    html: str,
    is_target_string: Union[str, Callable] = 'data.*mapping.*message',
    *,
    variable_name: str = 'window.__remixContext',
    json_pattern: str = '(\\{.*?\\});') -> dict
) -> dict:
```

In [29]:
from i2 import Sig

print(',\n'.join(str(Sig(parse_shared_chatGPT_chat)).split(',')))

(html: str,
 is_target_string: Union[str,
 Callable] = 'data.*mapping.*message',
 *,
 variable_name: str = 'window.__remixContext',
 json_pattern: str = '(\\{.*?\\});') -> dict


We did this so that we can easily maintain the parser's definition. 

At the time of writing this, we've noticed that the json we want is assigned to a 
variable named `window.__remixContext` and has fields `data`, `mapping`, and `message` in that order. 

But to be able to get that in the first place, we used `parse_shared_chatGPT_chat` with different parameters. 

We tried it with a chat we knew the contents of, and asked `parse_shared_chatGPT_chat` to find the json 
based on a substring we knew should be in there:

In [32]:
from oa.chats import url_to_html, parse_shared_chatGPT_chat

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
string_contained_in_conversation = 'apple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper'
chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html, is_target_string=string_contained_in_conversation)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

Let's get a truncated version of this dict, to make sure it's not too big to study...

In [21]:
from lkj import truncate_dict_values
from pprint import pprint
from pyperclip import copy


t = truncate_dict_values(chat_json_dict)
list(t)

['basename', 'future', 'isSpaMode', 'state']

In [22]:
# save this truncated dict to a file
import tempfile
import json
from pathlib import Path

tmp_filepath = tempfile.mktemp()
Path


len(json.dumps(t))

330325

## Parsing out the json

This is probably the most fragile part of the process. 
It's the part that broke previous tools I found to solve my problem.

The approach I'm taking here is to identify a part of the conversation that I know should be in the conversation, and using that to find what I'm looking for. 

So what I did here is copy the html of a conversation and used chatGPT (o1) to help me out. 
[Here's the conversation](https://chatgpt.com/share/6788d964-9e60-8013-a2b3-e191d567c8ad).

In [6]:
# Bootstrapping (when you don't know the structure, but you do know of a string that is contained in the conversation)

from oa.chats import parse_shared_chatGPT_chat, url_to_html

url = 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5'
string_contained_in_conversation = 'apple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper'

chat_html = url_to_html(url)
chat_json_dict = parse_shared_chatGPT_chat(chat_html, is_target_string=string_contained_in_conversation)
list(chat_json_dict)

['basename', 'future', 'isSpaMode', 'state']

In [33]:
# get a truncated version of the chat_json_dict (too big to study!)
from lkj import truncate_dict_values
import json 

truncated_json_dict = truncate_dict_values(chat_json_dict, max_list_size=1, max_string_size=90)
truncated_json_dict_string = json.dumps(truncated_json_dict, indent=2)
assert string_contained_in_conversation in truncated_json_dict_string, (
    "The string_contained_in_conversation is not in the truncated_json_dict_string. Increase the max_* parameters."
)

In [106]:
# Save to a file to go study it...

import tempfile
from pathlib import Path

tmp_filepath = tempfile.mktemp() + '.json'
Path(tmp_filepath).write_text(truncated_json_dict_string)
print(f"{len(truncated_json_dict_string)} characters written to {tmp_filepath}")

584063 characters written to /var/folders/mc/c070wfh51kxd9lft8dl74q1r0000gn/T/tmph5v0uvws.json


## Parsing out the conversation part of the json

In [81]:
from oa.chats import find_all_matching_paths_in_list_values
from dol import path_get

paths = find_all_matching_paths_in_list_values(truncated_json_dict, target_value=string_contained_in_conversation)
path = next(paths)
path

('state',
 'loaderData',
 'routes/share.$shareId.($action)',
 'serverResponse',
 'data',
 'mapping',
 '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 'message',
 'content',
 'parts')

So now we know the path to where our target string can be found:

In [82]:
path_get(truncated_json_dict, path)

['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']

We can also look around it to get a better sense of the relevant json structure

In [83]:
path_get(truncated_json_dict, path[:-1])

{'content_type': 'text',
 'parts': ['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']}

In [84]:
path_get(truncated_json_dict, path[:-2])

{'id': '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 'author': {'role': 'assistant', 'metadata': {}},
 'create_time': 1737020652.436654,
 'content': {'content_type': 'text',
  'parts': ['Hello World!  \napple, river, galaxy, chair, melody, shadow, cloud, puzzle, flame, whisper.']},
 'status': 'finished_successfully',
 'end_turn': True,
 'weight': 1,
 'metadata': {'finish_details': {'type': 'stop', 'stop_tokens': [200002]},
  'is_complete': True,
  'citations': [],
  'content_references': [],
  'message_type': None,
  'model_slug': 'gpt-4o',
  'default_model_slug': 'gpt-4o',
  'parent_id': '94469d89-ff37-48c1-bdab-44b27299d79b',
  'request_id': '902d2a5a0ed1e15c-MRS',
  'timestamp_': 'absolute',
  'shared_conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5'},
 'recipient': 'all'}

In [87]:
print(path[:-4])

('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')


A wild guess.
`('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')` is where the "turns" of the conversation are.

In [108]:
turns_path = ('state', 'loaderData', 'routes/share.$shareId.($action)', 'serverResponse', 'data', 'mapping')
turns_data = path_get(truncated_json_dict, turns_path)
list(turns_data)

['adff303b-75cc-493c-b757-605adadb8e56',
 '1473f2d9-ba09-4cd7-90c4-1452898676de',
 '40c5ed53-2b82-4e38-a4ec-dabf8f589553',
 '3b469b70-b069-4640-98af-5417491bb626',
 '94469d89-ff37-48c1-bdab-44b27299d79b',
 '38ee4a3f-8487-4b35-9f92-ddee57c25d0a',
 '01808c08-dc33-4932-bc12-64bd7e936760',
 'be4486db-894f-4e6f-bd0a-22d9d2facf69']

In [91]:
chat_data_path = turns_path[:-1]
list(path_get(truncated_json_dict, chat_data_path))

['title',
 'create_time',
 'update_time',
 'mapping',
 'moderation_results',
 'current_node',
 'conversation_id',
 'is_archived',
 'safe_urls',
 'default_model_slug',
 'disabled_tool_ids',
 'is_public',
 'linear_conversation',
 'has_user_editable_context',
 'continue_conversation_url',
 'moderation_state',
 'is_indexable',
 'is_better_metatags_enabled']

In [105]:
metadata = path_get(truncated_json_dict, chat_data_path)
metadata = {k: v for k, v in metadata.items() if k != 'mapping'}
metadata

{'title': 'Test Chat 1',
 'create_time': 1737020729.060687,
 'update_time': 1737020733.031014,
 'moderation_results': [],
 'current_node': 'be4486db-894f-4e6f-bd0a-22d9d2facf69',
 'conversation_id': '6788d539-0f2c-8013-9535-889bf344d7d5',
 'is_archived': False,
 'safe_urls': [],
 'default_model_slug': 'gpt-4o',
 'disabled_tool_ids': [],
 'is_public': True,
 'linear_conversation': [{'id': 'adff303b-75cc-493c-b757-605adadb8e56',
   'children': ['1473f2d9-ba09-4cd7-90c4-1452898676de']}],
 'has_user_editable_context': False,
 'continue_conversation_url': 'https://chatgpt.com/share/6788d539-0f2c-8013-9535-889bf344d7d5/continue',
 'moderation_state': {'has_been_moderated': False,
  'has_been_blocked': False,
  'has_been_accepted': False,
  'has_been_auto_blocked': False,
  'has_been_auto_moderated': False},
 'is_indexable': False,
 'is_better_metatags_enabled': True}

## Make some documentation for this part of the chat json

In [98]:
from oa import prompt_function


mk_json_field_documentation = prompt_function("""
You are a technical writer specialized in documenting JSON fields. 
Below is a JSON object. I'd like you to document each field in a markdown table.
The table should contain the name, description, and example value of each field.
                                              
The context is:
{context: just a general context}
                                              
Here's an example json object:

{example_json}
""")

mk_json_field_documentation

<function oa.tools.prompt_function.<locals>.ask_oa(example_json, *, context=' just a general context')>

In [110]:
metadata_field_docs = mk_json_field_documentation(
    json.dumps(turns_data, indent=2), 
    context="the conversation 'turns' of the json object holding a shared chatGPT conversation"
)
print(metadata_field_docs)

Here's the documentation for the provided JSON object, structured in a markdown table format:

| Field Name                                         | Description                                                                                     | Example Value                                                                         |
|---------------------------------------------------|-------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------|
| `<Conversation ID>`                               | Unique identifier for each turn in the conversation.                                          | `adff303b-75cc-493c-b757-605adadb8e56`                                             |
| `id`                                              | The unique identifier for the specific message/turn.                                         | `adff303b-75cc-493c-b757-605adadb8e56`   

In [99]:
metadata_field_docs = mk_json_field_documentation(
    json.dumps(metadata, indent=2), 
    context="metadata of a shared chatGPT conversation"
)
print(metadata_field_docs)

Here's the documentation for each field in the provided JSON object, formatted as a markdown table:

| Field Name                                | Description                                                                                          | Example Value                                                      |
|-------------------------------------------|------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------|
| `title`                                   | The title of the shared chat GPT conversation.                                                      | "Test Chat 1"                                                    |
| `create_time`                             | The timestamp when the conversation was created, represented in seconds since the Unix epoch.      | 1737020729.060687                                                |
| `update_time`                    

In [12]:
import re
import json
from typing import Callable, Optional

def raise_run_time_error(msg: str):
    raise RuntimeError(msg)

def extract_json_dict(
    string: str,
    object_filter: Callable,
    *,
    decoder: json.JSONDecoder = json.JSONDecoder(),
    not_found_callback: Optional[
        Callable
    ] = lambda string, object_filter: raise_run_time_error(
        "Object not found in string"
    ),
) -> dict:
    """
    Searches `string` from the beginning, attempting to decode consecutive
    JSON objects using `decoder.raw_decode`. When an object satisfies
    `object_filter`, returns it as a Python dictionary.
    """
    pos = 0
    while pos < len(string):
        try:
            obj, pos = decoder.raw_decode(string, pos)
            if object_filter(obj):
                return obj
        except json.JSONDecodeError:
            pos += 1
    return not_found_callback(string, object_filter)

def parse_shared_chatGPT_chat(
    html: str, 
    string_contained_in_conversation: str
) -> dict:
    """
    Locates the big JSON structure assigned to window.__remixContext, 
    checks that the conversation portion includes `string_contained_in_conversation`,
    and returns the JSON as a dict.
    """
    # Regex pattern to capture the object assigned to window.__remixContext = {...};
    pattern = r"window\.__remixContext\s*=\s*(\{.*?\});"
    match = re.search(pattern, html, flags=re.DOTALL)
    if not match:
        raise RuntimeError("Could not locate the JSON assigned to window.__remixContext")

    # Extract the raw JSON text (removing the trailing semicolon if needed)
    raw_json_text = match.group(1).strip()

    # We'll define a filter that checks if the conversation data includes the target string.
    # Because the JSON is quite large, we can simply check if the substring is present in the raw text:
    # but if we want to be more precise, we can parse first and only return if there's conversation data.
    def conversation_filter(obj):
        # Convert to string once more, or deeply check "routes/share.$shareId.($action)" for the substring.
        # For simplicity, do a textual check on the entire string representation. 
        # If needed, we can refine to parse a specific location in obj.
        return string_contained_in_conversation in json.dumps(obj, ensure_ascii=False)

    # Now parse the JSON, returning only if the filter passes
    extracted_dict = extract_json_dict(
        raw_json_text,
        object_filter=conversation_filter,
    )

    return extracted_dict


t = parse_shared_chatGPT_chat(html, existing_substring)

['basename', 'future', 'isSpaMode', 'state']